# Left Eye (LE) - Luminance Estimation Model Training

This notebook provides a clean, organized pipeline for training models for Left Eye luminance estimation.
It follows a modular structure with clear sections for data loading, model architecture, and training.

## Experiment Overview
- **Dataset**: Left Eye images with luminance labels
- **Task**: Regress two continuous values (luminance coordinates)
- **Approaches**: Baseline model, Transfer Learning, From Scratch

## Section 1: Setup & Imports

In [ ]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path for module imports
# sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Import custom modules
from config import get_config
from data_loader import load_images, CustomDataGenerator, get_augmentation_generator
from model_architecture import create_and_compile_model
from training_utils import train_model, plot_training_results, save_history_to_json, get_callbacks
from system_monitoring import print_system_info

print("All imports successful!")

## Section 2: System Information & Configuration

In [ ]:
# Print system information
print_system_info()

# Get experiment configuration
experiment = 'LE'  # Change to 'Face' or 'RE' for other experiments
config = get_config(experiment)

print(f"\nExperiment: {config['experiment_info']['name']}")
print(f"Image Size: {config['image_size']}")
print(f"Batch Size: {config['batch_size']}")
print(f"Epochs: {config['epochs']}")
print(f"Learning Rate: {config['learning_rate']}")

## Section 3: Load Data

In [ ]:
# Get experiment info
exp_info = config['experiment_info']

print("Loading baseline training data...")
train_samples, train_labels = load_images(
    exp_info['baseline'],
    image_size=config['image_size']
)

print("\nLoading test data...")
test_samples, test_labels = load_images(
    exp_info['test'],
    image_size=config['image_size']
)

print(f"\nTrain samples shape: {train_samples.shape}")
print(f"Train labels shape: {train_labels.shape}")
print(f"Test samples shape: {test_samples.shape}")
print(f"Test labels shape: {test_labels.shape}")

## Section 4: Create Data Generators

In [ ]:
# Create augmentation generator
datagen = get_augmentation_generator()

# Create custom data generators
train_generator = CustomDataGenerator(
    train_samples, train_labels,
    batch_size=config['batch_size'],
    shuffle=True,
    datagen=datagen
)

test_generator = CustomDataGenerator(
    test_samples, test_labels,
    batch_size=config['batch_size'],
    shuffle=False,
    datagen=None
)

print(f"Training batches per epoch: {len(train_generator)}")
print(f"Test batches per epoch: {len(test_generator)}")

## Section 5: Train Baseline Model

In [ ]:
# Create model
model = create_and_compile_model(
    input_shape=(config['image_size'], config['image_size'], 3),
    dense_units=96,
    dropout_rate=config['dropout_rate'],
    learning_rate=config['learning_rate']
)

print("Model created and compiled!")
model.summary()

In [ ]:
# Get callbacks
callbacks = get_callbacks(
    patience_early_stopping=config['patience_early_stopping'],
    patience_reduce_lr=config['patience_reduce_lr'],
    factor_reduce_lr=config['factor_reduce_lr'],
    min_learning_rate=config['min_learning_rate']
)

# Train model
results = train_model(
    model,
    train_generator,
    test_generator,
    epochs=config['epochs'],
    callbacks=callbacks,
    model_name="participant_baseline"
)

history = results['history']
training_time = results['time']

print(f"\nTraining completed in {training_time['hours']:.2f} hours")

## Section 6: Visualize Training Results

In [ ]:
# Plot training results
plot_training_results(history, model_name="(participant_baseline)")

In [ ]:
# Summary statistics
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_mae = history.history['mae']
val_mae = history.history['val_mae']

print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)
print(f"Epochs trained: {len(train_loss)}")
print(f"\nFinal Training Loss: {train_loss[-1]:.6f}")
print(f"Final Validation Loss: {val_loss[-1]:.6f}")
print(f"\nBest Validation Loss: {min(val_loss):.6f} (epoch {np.argmin(val_loss)+1})")
print(f"Best Validation MAE: {min(val_mae):.6f} (epoch {np.argmin(val_mae)+1})")
print(f"\nTraining Time: {training_time['hours']:.2f} hours")
print("="*60)

## Section 7: Save Model & Results

In [ ]:
# Save model
model_name = 'participant_baseline'
model.save(f'{model_name}.keras')
print(f"Model saved as {model_name}.keras")

# Save training history
save_history_to_json(history, f'{model_name}.json')

## Section 8: Transfer Learning with Different Dataset Sizes

Fine-tune the baseline model with different amounts of Ola-augmented data.

In [ ]:
# Save model
model_name = 'participant_baseline'
model.save(f'{model_name}.keras')
print(f"Model saved as {model_name}.keras")

# Save training history
save_history_to_json(history, f'{model_name}.json')

In [ ]:
# Create generators for transfer learning
ola_train_gen = CustomDataGenerator(
    ola_train_samples, ola_train_labels,
    batch_size=config['batch_size'],
    shuffle=True
)

ola_test_gen = CustomDataGenerator(
    test_samples, test_labels,
    batch_size=config['batch_size'],
    shuffle=False
)

# Load baseline model for fine-tuning
from tensorflow.keras.models import load_model
model_ft = load_model(f'{model_name}.keras')

# Fine-tune
callbacks_ft = get_callbacks(
    patience_early_stopping=config['patience_early_stopping'],
    patience_reduce_lr=config['patience_reduce_lr'],
    factor_reduce_lr=config['factor_reduce_lr'],
    min_learning_rate=config['min_learning_rate']
)

results_ft = train_model(
    model_ft,
    ola_train_gen,
    ola_test_gen,
    epochs=config['epochs'],
    callbacks=callbacks_ft,
    model_name=f"LE_Transfer_Ola_{dataset_key}"
)

history_ft = results_ft['history']
print(f"\nTransfer learning completed")

In [ ]:
# Plot transfer learning results
plot_training_results(history_ft, model_name=f"(LE Transfer - {dataset_key} Ola)")

## Section 9: From-Scratch Training with Ola Data

In [ ]:
# Create new model for from-scratch training
model_scratch = create_and_compile_model(
    input_shape=(config['image_size'], config['image_size'], 3),
    dense_units=96,
    dropout_rate=config['dropout_rate'],
    learning_rate=config['learning_rate']
)

callbacks_scratch = get_callbacks(
    patience_early_stopping=config['patience_early_stopping'],
    patience_reduce_lr=config['patience_reduce_lr'],
    factor_reduce_lr=config['factor_reduce_lr'],
    min_learning_rate=config['min_learning_rate']
)

results_scratch = train_model(
    model_scratch,
    ola_train_gen,
    ola_test_gen,
    epochs=config['epochs'],
    callbacks=callbacks_scratch,
    model_name=f"participant_scratch_{dataset_key}"
)

history_scratch = results_scratch['history']
print(f"\nFrom-scratch training completed")

In [ ]:
# Create generators for transfer learning
ola_train_gen = CustomDataGenerator(
    ola_train_samples, ola_train_labels,
    batch_size=config['batch_size'],
    shuffle=True
)

ola_test_gen = CustomDataGenerator(
    test_samples, test_labels,
    batch_size=config['batch_size'],
    shuffle=False
)

# Load baseline model for fine-tuning
from tensorflow.keras.models import load_model
model_ft = load_model(f'{model_name}.keras')

# Fine-tune
callbacks_ft = get_callbacks(
    patience_early_stopping=config['patience_early_stopping'],
    patience_reduce_lr=config['patience_reduce_lr'],
    factor_reduce_lr=config['factor_reduce_lr'],
    min_learning_rate=config['min_learning_rate']
)

results_ft = train_model(
    model_ft,
    ola_train_gen,
    ola_test_gen,
    epochs=config['epochs'],
    callbacks=callbacks_ft,
    model_name=f"participant_transfer_{dataset_key}"
)

history_ft = results_ft['history']
print(f"\nTransfer learning completed")

## Section 10: Conclusions & Notes

- **Baseline Model**: Trained on data without Ola augmentation
- **Transfer Learning**: Fine-tunes baseline with Ola-augmented data
- **From-Scratch**: Trains new model directly on Ola-augmented data

Compare the results to determine which approach works best for your dataset.